# Machine Learning

### Was ist Machine Learning?

Machine Learning, im Deutschen maschinelles Lernen genannt, ist ein Teilbereich von künstlicher Intelligenz.
Ein Machine Learning Algorithmus versucht aus Beispieldaten Zusammenhänge zu finden, womit es dann eine Vorhersage über neue Daten machen kann.
Diese Algorithmen sind nicht explizit programmiert, sondern entwickeln sind anhand der Daten ständig weiter.

Für eine Vorhersage über neue Daten brauchen wir ein '**target**', also ein Ziel. In unserem Beispiel wird das der **Urbanisierungsgrad** von unseren Gemeinden sein.

Es gibt unterschiedliche Arten des maschinellen Lernens. Wir werden uns hier mit **überwachtem Lernen** beschäftigen, wo wir unseren Algorithmen die 'richtige Lösung' bei den Beispieldaten verraten.
Weiters unterscheidet man zwischen **Klassifikation** und **Regression**. Bei einer Klassifikation wollen wir einen kategorischen Wert schätzen, während wir bei der Regression versuchen, so nahe wie möglich auf den richtigen numerischen Wert zu kommen.

Da wir den Urbanisierungsgrad unserer Gemeinden vorhersagen wollen und überwachtes Lernen benutzen, müssen wir 'Urbanisierungsgrad' zu unserem Datensatz hinzufügen. Der Urbanisierungsgrad ist mit den Werten 1, 2 und 3 kodiert. Diese repräsentieren nicht numerische, sondern kategorische Werte, weil sie dem Grad der Urbanisierung entsprechen (1: Hoch, 2: Mittel, 3: Niedrig). Wir werden die Werte auf Text umändern, damit klar ist, dass es sich hier um Klassifikation handelt.

In [ ]:
# Import der Pandas Library
import pandas as pd

# Import des zuvor gespeicherten Datensatzes
gem_daten_sauber = pd.read_csv('gem_daten_sauber.csv')

gem_daten_sauber.head()

### Bundesland hinzufügen

In [ ]:
#Idee: ML für Bundesländer: Gemeindekennzahl bzw Gemeindecode ignorieren, "Bundaesland" feld erstellen mit erster Ziffer (1-9).
#Oder ist es besser, Kategorien zu erstellen? Zahlen 1-9 könnten als numerische Interpretation falsch verwendet werden.

gem_daten_sauber['Bundesland'] = gem_daten_sauber['Gemeindekennziffer'].astype(str).str[0]
gem_daten_sauber['Bundesland'] = gem_daten_sauber['Bundesland'].astype(int)

gem_daten_sauber.head()

In [ ]:
gem_daten_sauber.dtypes

In [ ]:
d_bundesland = {1:'Burgenland', 2:'Kärnten', 3:'Niederösterreich', 4:'Oberösterreich',
                5:'Salzburg', 6:'Steiermark', 7:'Tirol', 8:'Vorarlberg', 9:'Wien'}
gem_daten_sauber['Bundesland'].replace(d_bundesland, inplace=True)

### Urbanisierungsgrad hinzufügen

Quelle: [Statistik Austria](https://www.statistik.at/web_de/klassifikationen/regionale_gliederungen/stadt_land/index.html)

Unser neuer Datensatz besteht aus 3 den Spalten *GKZ*, *CODE* und *TXT.

* Über die Spalte *GKZ* können wir diesen Datensatz mit unseren sauberen Gemeindedaten (via *Gemeindekennziffer*) verbinden.
* Die Spalte *TXT*, wo die Kodierung in Textform gespeichert ist, werden wir nicht brauchen,
* weil die numerische Kodierung index *CODE* in Text umwandeln werden.

In [ ]:
urbanisierungsgrad = pd.read_csv('gemeinden_urbanisierungsgrad.csv', sep=';')

urbanisierungsgrad.head()

In [ ]:
# Zusammenführen der beiden Datensätze über die Gemeindekennziffern

gem_daten_sauber = gem_daten_sauber.join(urbanisierungsgrad.set_index('GKZ'), on='Gemeindekennziffer')


In [ ]:
# Visuelle Überprüfung des neuen, kombinierter Datensatzes:
gem_daten_sauber.head()

***Achtung:*** Bei manchen Codezellen muss man aufpassen, dass man sie nicht mehrmals ausführt. In der folgenden Codezelle werden nämlich Spalten aus dem Datensatz entfernt. Führt man diese Zelle nochmal aus, kommt eine Fehlermeldung, dass die Spalten, die man versucht hat zu entfernen, nicht gefunden wurden. Das passiert, weil diese Spalten schon bei der ersten Ausführung von der Codezelle entfernt wurden und nun nicht mehr existieren. Falls du auf so ein Problem stoßt, kannst du einfach alle Codezellen von oben neu ausführen.

In [ ]:
# Entfernung von Spalten die wir (fürs Machine-Learning) 
# nicht mehr brauchen:
df_norm.drop(columns=['PLZ des Gem.Amtes', 'weitere Postleitzahlen', 'TXT'], inplace=True)

# Die Spalte 'CODE' nennen wir nun 'Urbanisierungsgrad'
df_norm.rename(columns= {'CODE' : 'Urbanisierungsgrad'}, inplace=True)

# Leserfreundliche Umbenennung der Urbanisierungsgrade:
# 1 bedeutet 'Hoch', 2 'Mittel' und 3 ist 'Niedrig'
d_urbanisierungsgrad = {1:'Hoch', 2:'Mittel', 3:'Niedrig'}
df_norm['Urbanisierungsgrad'].replace(d_urbanisierungsgrad, inplace=True)

# Übersicht:
df_norm.head()

Wir wollen unserem Machine Learning Algorithmus also Daten geben, die er noch nicht kennt, damit es auf diesen unbekannten Daten den Urbanisierungsgrad schätzt, welches entweder 'Hoch', 'Mittel' oder 'Niedrig' sein wird.

Dafür wollen wir einen kleinen Teil unserer Daten (wir entscheiden uns hier für 2%) raus nehmen und zur Seite stellen, damit dieser am Ende für Schätzungen benutzt werden kann.

In [ ]:
# 2% der Daten ausschließen damit man sie nur zum Schätzen benutzt:
ml_datensatz = df_norm.sample(frac=0.98, random_state=101)
ungesehene_daten = df_norm.drop(ml_datensatz.index)

ml_datensatz.reset_index(drop=True, inplace=True)
ungesehene_daten.reset_index(drop=True, inplace=True)

# Neue Größe unserer Daten: (Anzahl an Zeilen, Anzahl der Spalten)
print('Daten für die Modellierung: ' + str(ml_datensatz.shape))
print('Ungesehene Daten für Schätzungen: ' + str(ungesehene_daten.shape))

In [ ]:
# Überblick über die Daten, die wir für Machine Learning verwenden:
ml_datensatz.head()

In [ ]:
# Überblick über die Daten, die wir zum Schätzen verwenden (42 insgesamt):
ungesehene_daten.head()

# Setup

Als ersten Schritt müssen wir '**setup**' ausführen. Gehen wir die wichtigsten Parameter durch: 
* **data**: Wir müssen den Datensatz, den wir verwenden wollen, angeben.
* **target**: Die Spalte, welches wir als Ziel ausgesucht haben und vorhersagen wollen.
* **ignore_features**: Spalten die ignoriert werden sollen. Wir entfernen 'Gemeindekennziffer', 'Gemeindename' und 'Gemeindecode', weil diese keine Information über den Urbanisierungsgrad bieten und die Algorithmen potenziell schlechter werden. Zusätzlich ignorieren wir 'Einwohner' und 'Arbeitsstätten', weil diese zu viel über den Urbanisierungsgrad verraten könnten. 
* **train_size**: Wir unterscheiden zwischen **Trainingsdaten** und **Testdaten**. Trainingsdaten sind die Beispieldaten, die unsere Algorithmen benutzen um zu lernen, während man danach auf den Testdaten die Genauigkeit von den Algorithmen evaluiert. Wir haben uns hier für 70% Trainingsdaten und 30% Testdaten entschieden.

Wenn man auf **3: Label Encoded** schaut, sieht man, dass die verschiedenen Urbanisierungsgrade automatisch wieder als Zahlen kodiert wurden.

***Hinweis:*** Manche der kommenden Codezellen können einige Minuten dauern!

In [ ]:
# Für Machine Learning benutzen wir PyCaret als Tool und benötigen die Algorithmen für Klassifikation

from pycaret.classification import *
exp = setup(data=ml_datensatz, target='Urbanisierungsgrad', ignore_features=['Gemeindekennziffer', 'Gemeindename', 'Gemeindecode', 'Einwohner', 'Arbeitsstätten', 'Bundesland'], silent=True, session_id=101, train_size=0.70)

# Modelltest

Als zweiten Schritt wollen wir nun die verschiedenen Machine Learning Algorithmen miteinander vergleichen (**compare_models**). Es werden ein paar von ihnen aufgrund der langen Rechenzeit exkludiert.

Die Anzahl an **folds** muss bestimmt werden: 'fold=3' bedeutet, dass wir 3-mal 70% (siehe: setup) unserer Daten als Trainingsdaten verwenden. Dabei werden die Trainingsdaten jedes Mal zufällig ausgesucht und sind daher unterschiedlich.
Der Grund, wieso wir folds verwenden, ist damit wir ein sogenanntes **overfitting** vermeiden. Overfitting bedeutet, dass unser Algorithmus die Trainingsdaten zu genau gelernt hat, wodurch Probleme entstehen können, wie zum Beispiel dass es neue Daten komplett falsch schätzt, wenn sie sich zu sehr von den Trainingsdaten unterscheiden.

In [ ]:
# Wir vergleichen die ML-Algorithmen miteinander:
best = compare_models(fold=3, exclude=['lightgbm'])

Die gelben Markierungen in der Tabelle heben die besten Werte hervor. In diesem Beispiel interessiert uns **Accuracy**, die Genauigkeit vom Algorithmus, am meisten.

Wir sehen, dass das *Random Forest* Modell eine sehr hohe Genauigkeit *und* eine niedrige Laufzeit (**TT (Sec)**) hat und wollen den als unser Modell nehmen. Dafür brauchen wir den Befehl *create_model*.

In [ ]:
# Wähle Random Forest Classifier als Modell, mit 3 folds
selected_model = create_model('rf', fold=3)

Mit **tune_model** können wir einige der Ergebnisse weiter verbessern.

***Hinweis:*** Die Ausführung dauert länger.

In [ ]:
# Mit tune_model können wir unser ausgesuchtes Modell
# (Random Forest Classifier) weiter verbessern.

tuned_model = tune_model(selected_model, fold=3)

## Auswertung der Vorhersagequalität

Es existieren viele Visualisierungen, die einem helfen können, die Ergebnisse vom Algorithmus besser zu verstehen. Wir fokussieren uns hier auf drei davon.

### Confusion Matrix

Hier siehst du die Anzahl der Daten, die richtig bzw. falsch klassifiziert wurden. Auf der x-Achse sind die Urbanisierungsgrade, die vom Algorithmus geschätzt wurden, während sich auf der y-Achse die tatsächlichen Urbanisierungsgrade von den Daten befinden.

**Errinnerung:** Urbanisierungsgrad: 'Hoch' wurde als '0' kodiert, 'Mittel' mit 1 und 'Niedrig' mit 2.

Vielleicht hast du bemerkt, dass die Anzahl der Daten, die hier vorkommen, weniger ist als die Größe deines Datensatzes. Das liegt daran, dass nur 30% der Daten als Testdaten verwendet werden.

In [ ]:
# Confusion Matrix
plot_model(tuned_model, plot='confusion_matrix')

### Class Prediction Error

Das zeigt im Prinzip dieselbe Information wie unsere Confusion Matrix aber in Form eines Balkendiagrammes.

Auf der x-Achse ist der echte Urbanisierungsgrad und auf der y-Achse die Anzahl an Schätzungen.

Man merkt, dass es sehr wenige Gemeinden mit Urbanisierungsgrad 'Hoch' gibt, sie aber trotzdem alle richtig geschätzt wurden. Ab und zu werden Gemeinden, die Urbanisierungsgrad 'Mittel' oder 'Niedrig' haben, miteinander verwechselt. Wir sehen auch, dass die meisten Gemeinden einen niedrigen Urbanisierungsgrad haben.

In [ ]:
# Fehler bei der Klassifizierung von den unterschiedlichen Urbanisierungsgraden
plot_model(tuned_model, plot='error')

### Feature Importance

Beim Feature Importance Plot sieht man, welche Spalten aus unserem Gemeindedatensatz den größten Einfluss zur Bestimmung des Urbanisierungsgrads haben. Gibt es welche, die du nicht erwartet hast?

In [ ]:
plot_model(tuned_model, plot='feature')

Wien als Bundesland scheint einen auffällig hohen Einfluss auf unser Algorithmus zu haben. Wieso könnte das sein? Am besten, wir schauen uns an, welche Gemeinden denn überhaupt einen hohen Urbanisierungsgrad haben.

In [ ]:
# Wir suchen nach allen Datensätzen, die einen hohen Urbanisierungsgrad haben

urban_hoch = gem_daten_sauber[gem_daten_sauber['Urbanisierungsgrad'] == 'Hoch']
urban_hoch.head(20)

Es sind fast alle Gemeinden mit hohem Urbanisierungsgrad in Wien. Wenn du Lust hast, kannst du versuchen bei **setup** **Bundesland** zu **exkludieren** und zu schauen, ob sich was an der Genauigkeit (insbesondere bei hoher Urbanisierung) vom Algorithmus ändert. 

# Abschluss

Wenn wir mit unserem Modell zufrieden sind, wollen wir es mit **finalize_model** finalisieren. Es werden jetzt die Testdaten auch zu Trainingsdaten gemacht.


In [ ]:
final_model = finalize_model(tuned_model)

# Neue Daten vorhersagen

Wir haben uns am Anfang 2% der Daten herausgenommen, damit wir am Ende mit unserem fertigen Machine Learning Modell ein paar Schätzungen machen können. Mit **predict_model** können wir nun unser Algorithmus auf den ungesehenen Daten verwenden. 

Du wirst erkennen, dass neben 'Urbanisierungsgrad' eine neue Spalte, nämlich **Label**, dazugekommen ist. Das ist die Klassifizierung vom Algorithmus, während 'Urbanisierungsgrad' der echte Wert von unserer Spalte ist. 
In Zeile 12 von der Tabelle siehst du beispielsweise, dass der Urbanisierungsgrad von Katzelsdorf eigentlich 'Mittel' ist, aber unser Algorithmus es fälschlicherweise als 'Niedrig' eingeschätzt hat.

In [ ]:
geschaetzte_daten = predict_model(final_model, data=ungesehene_daten)
geschaetzte_daten.head(42)